In [ ]:
!pip install opencv-python-headless==4.2.0.34
!pip install pytransform3d==1.1.1

## Notes

- Camera CS: Z - forward, X - right, Y - down

In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import os
from scipy.spatial.transform import Rotation as R

ROOT_DIR = '../'

In [ ]:
DATASET_DIR = os.path.join(ROOT_DIR, 'data/KITTY/dataset')
POSES_DIR = os.path.join(DATASET_DIR, 'poses')
SEQUENCE_IDX = '00'
SEQUENCE_DIR = os.path.join(DATASET_DIR, 'sequences', SEQUENCE_IDX)

In [ ]:
POSE_PATH = os.path.join(POSES_DIR, f'{SEQUENCE_IDX}.txt')
TIMES_PATH = os.path.join(SEQUENCE_DIR, 'times.txt')

poses_data = np.fromfile(POSE_PATH, sep=' ')
poses_data = poses_data.reshape((-1, 3, 4))

times_data = np.fromfile(TIMES_PATH, sep='\n')

In [ ]:
times_data.shape

In [ ]:
poses_data.shape

In [ ]:
pose_0 = poses_data[0]
pose_0

In [ ]:
pose_1 = poses_data[1]
pose_1

In [ ]:
pose_10 = poses_data[10]
pose_10

In [ ]:
inv_pose_rot_10 = np.linalg.inv(pose_10[:3,:3])
inv_pose_rot_10

In [ ]:
def get_transofrmation(pose1, pose2):
    rot_pose1 = pose1[:3,:3]
    rot_pose2 = pose2[:3,:3]
    inv_rot_pose1 = np.linalg.inv(rot_pose1)
    inv_rot_pose2 = np.linalg.inv(rot_pose2)
    trans_pose1 = pose1[:,3]
    trans_pose2 = pose2[:,3]
    
    r2_invr1 = rot_pose2.dot(inv_rot_pose1)
    trans = trans_pose2 - (r2_invr1.dot(trans_pose1))
    rot = r2_invr1
    
    trans = trans[..., np.newaxis]
    return np.hstack((rot, trans))

def get_transofrmation2(pose1, pose2):
    full_pose1 = np.vstack((pose1, [0,0,0,1]))
    full_pose2 = np.vstack((pose2, [0,0,0,1]))

    inv_pose1 = np.linalg.inv(full_pose1)
    return full_pose2.dot(inv_pose1)[:3,:]

In [ ]:
transform1 = get_transofrmation(poses_data[0], poses_data[1])
transform1

In [ ]:
transform2 = get_transofrmation2(poses_data[0], poses_data[1])
transform2

In [ ]:
np.all(transform1 == transform2)

In [ ]:
r = R.from_matrix(transform1[:,:3])
r.as_quat()

In [ ]:
r.as_euler('zyx', degrees=True)

In [ ]:
turn = get_transofrmation(poses_data[90], poses_data[100])
r = R.from_matrix(turn[:,:3])
r.as_euler('zyx', degrees=True)

In [ ]:
points = []
for pose in poses_data[:90]:
    points.append(
        (pose[0,3], pose[1,3], pose[2,3])
    )
    
points = np.array(points) 
plt.figure(figsize=(9,9))
plt.plot(points[:,0], points[:,2])